<a href="https://colab.research.google.com/github/samuelhtampubolon/SDPM2025/blob/main/Unsupervised_Anomaly_Detection_for_Fraud_Using_PyTorch_Autoencoder_with_Synthetic_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Generate synthetic data (train on normal, test with fraud)
np.random.seed(42)
normal = np.random.normal(0, 1, (950, 2))
fraud = np.random.normal(5, 1, (50, 2))
X_test = np.vstack((normal[:50], fraud))  # Test set

X_normal = torch.from_numpy(normal).float()
X_test_tensor = torch.from_numpy(X_test).float()

In [2]:
# Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Linear(2, 5)
        self.decoder = nn.Linear(5, 2)

    def forward(self, x):
        return self.decoder(torch.relu(self.encoder(x)))

model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [3]:
# Training on normal data
for epoch in range(100):
    optimizer.zero_grad()
    out = model(X_normal)
    loss = criterion(out, X_normal)
    loss.backward()
    optimizer.step()

In [4]:
# Detect anomalies by reconstruction error
with torch.no_grad():
    errors = torch.mean((model(X_test_tensor) - X_test_tensor)**2, dim=1)
threshold = errors.mean() + 3 * errors.std()
anomalies = errors > threshold
print("Detected frauds:", anomalies.sum().item())

Detected frauds: 1
